https://huggingface.co/docs/transformers/tasks/language_modeling
Link to follow the tutorial

In [1]:
import pandas as pd
import numpy as np
import gc
from transformers import Adafactor
import tqdm
from tqdm.notebook import tqdm_notebook
import torch
import evaluate 
import wandb
import math

In [2]:
wandb.init(project="LongT5", resume=True)

wandb: Currently logged in as: fayz. Use `wandb login --relogin` to force relogin


In [3]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_json('/home/beyond-data/Documents/Faiza/Notebooks for Finetuning/Data/train.json')
test = pd.read_json('/home/beyond-data/Documents/Faiza/Notebooks for Finetuning/Data/test.json')
result_df = pd.read_csv('LongT5results.csv')
train['input'] = train['Question'] + train['document']
test['input'] = test['Question'] + test['document']
del train['Question']
del test['Question']
del train['document']
del test['document']
#train
for i in tqdm_notebook(range(len(train['input']))):
  if len(train['input'][i]) > 4096:
    train['input'][i] = train['input'][i][:4096]
  if len(train['Answer'][i]) > 512:
    train['Answer'][i] = train['Answer'][i][:512]
    
#test
for i in tqdm_notebook(range(len(test['input']))):
  if len(test['input'][i]) > 4096:
    test['input'][i] = test['input'][i][:4096]
  if len(test['Answer'][i]) > 512:
    test['Answer'][i] = test['Answer'][i][:512]

  0%|          | 0/46753 [00:00<?, ?it/s]

  0%|          | 0/6676 [00:00<?, ?it/s]

In [5]:
result_df

,Unnamed: 0,epoch,training_loss,validation_loss,perplexity,rouge1,rouge2,rougeL,rougeLsum
0,0,1,2.333190,3.573771,35.650787,NaN,NaN,NaN,NaN
1,1,2,2.041033,2.604893,13.529783,NaN,NaN,NaN,NaN
2,2,3,1.664399,2.583782,13.247143,NaN,NaN,NaN,NaN
3,3,4,1.331023,2.759908,15.798392,NaN,NaN,NaN,NaN
4,4,5,1.483228,2.730522,15.340896,NaN,NaN,NaN,NaN
5,5,6,1.051048,3.059094,21.308252,NaN,NaN,NaN,NaN
6,6,7,0.820959,3.226067,25.180420,NaN,NaN,NaN,NaN
7,7,8,0.663037,3.432674,30.959332,NaN,NaN,NaN,NaN
8,8,9,0.553959,3.836314,46.354313,NaN,NaN,NaN,NaN
9,9,10,0.455134,4.043764,57.040623,NaN,NaN,NaN,NaN


In [6]:
del result_df['Unnamed: 0']

In [7]:
epoch = len(result_df['epoch'])+1

In [8]:
epoch

16

In [9]:
from transformers import AutoTokenizer
from transformers import LongT5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")
if len(result_df['epoch'])>0:
    model = LongT5ForConditionalGeneration.from_pretrained("LongT5forQAS"+str(epoch-1)).to(dev)
else:
    model = LongT5ForConditionalGeneration.from_pretrained("google/long-t5-tglobal-base").to(dev)

In [11]:
import gc
from transformers import Adafactor

seed = 1287
np.random.seed(seed)
train_df=train
batch_size=1
num_of_batches=len(train_df)/batch_size
print(num_of_batches)
gc.collect()

optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

46753.0


In [18]:
del result_df['rouge1']
del result_df['rouge2']
del result_df['rougeL']
del result_df['rougeLsum']

In [19]:
result_row = []
result_row.append(epoch)
result_row.append(running_loss)
result_row.append(3.1126711569477314)
result_row.append(0.0)
result_df.loc[len(result_df)]=result_row
result_df.to_csv('LongT5results.csv')

In [20]:
def model_validation(model,epoch, running_loss):
    model.eval()
    bs = 1
    val_loss=0
    ppl=0
    test_num_of_batches=len(test)/bs
    predictions = []
    torch.cuda.empty_cache()
    gc.collect()
    print("Validating the model")
    for i in tqdm_notebook(range(int(test_num_of_batches))):
        ib=[]
        lb = []
        new_df=test[i*bs:i*bs+bs]
        for indx,row in new_df.iterrows():
          input =row['input']
          ib.append(input)
          lb.append(row['Answer'])
          torch.cuda.empty_cache()
        ib=tokenizer.batch_encode_plus(ib,truncation=True,padding=True,max_length=4096,return_tensors='pt')["input_ids"].to(dev)
        lb=tokenizer.batch_encode_plus(lb,truncation=True,padding=True,max_length=512,return_tensors='pt')["input_ids"].to(dev)
        optimizer.zero_grad()
        outputs = model(input_ids=ib,labels=lb)
        loss = outputs.loss
        val_loss += loss.item()
    val_loss = val_loss/test_num_of_batches
    print("Validation loss: ", val_loss)
    wandb.log({"Training loss":running_loss,"Validation loss": val_loss})
    try:
        val_loss = np.mean(val_loss)
        ppl = math.exp(val_loss)
        wandb.log({"Perplexity": ppl})
    except:
        pass
    result_row = []
    result_row.append(epoch)
    result_row.append(running_loss)
    result_row.append(val_loss)
    result_row.append(ppl)
    result_df.loc[len(result_df)]=result_row
    result_df.to_csv('LongT5results.csv')

In [22]:
epoch=17

In [ ]:
model.train()

loss_per_10_steps=[]
num_of_epochs=30
for epoch in range(epoch,num_of_epochs+1):
  print('Running epoch: {}'.format(epoch))
  
  running_loss=0

  out = display(progress(1, num_of_batches+1), display_id=True)
  for i in tqdm_notebook(range(int(num_of_batches))):
    inputbatch=[]
    labelbatch=[]
    new_df=train_df[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      input = 'LFQA: '+row['input']+'</s>' 
      labels = row['Answer']+'</s>'   
      inputbatch.append(input)
      labelbatch.append(labels)
      torch.cuda.empty_cache()
    inputbatch=tokenizer.batch_encode_plus(inputbatch,truncation=True,padding=True,max_length=4096,return_tensors='pt')["input_ids"]
    labelbatch=tokenizer.batch_encode_plus(labelbatch,truncation = True,padding=True,max_length=512,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(dev)
    labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()
    #torch.empty_cache()
    gc.collect()
  running_loss=running_loss/int(num_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
  wandb.watch(model)
  model.save_pretrained('LongT5forQAS'+str(epoch))
  model_validation(model, epoch, running_loss)

Running epoch: 17


  0%|          | 0/46753 [00:00<?, ?it/s]

/home/beyond-data/anaconda3/envs/longt5/lib/python3.7/site-packages/transformers/modeling_utils.py:770: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error resolved after 0:00:08.262894, resuming normal operation.


Epoch: 17 , Running loss: 0.8031695800611922
Validating the model


  0%|          | 0/6676 [00:00<?, ?it/s]

...

Epoch: 25 , Running loss: 0.12818549137432136
Validating the model


  0%|          | 0/6676 [00:00<?, ?it/s]

Validation loss:  5.005867832935211
Running epoch: 26


  0%|          | 0/46753 [00:00<?, ?it/s]

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.


In [17]:
result_df = pd.DataFrame(columns = ['epoch','training_loss','validation_loss','perplexity'])

In [18]:
result_df

,epoch,training_loss,validation_loss,perplexity


In [25]:
wandb.log({"Perplexity":math.exp(np.mean(3.5737712138816224))})

In [22]:
result_row=[]

In [23]:
result_row.append(epoch)
result_row.append(2.33319)
result_row.append(3.5737712138816224)
result_row.append(math.exp(np.mean(3.5737712138816224)))
result_df.loc[len(result_df)]=result_row
result_df.to_csv('LongT5results.csv')

In [11]:
result_df.loc[len(result_df)]=result_row[:8]
result_df.to_csv('LongT5results.csv')

NameError: name 'result_row' is not defined

In [10]:
model_validation(model,epoch,2.33319)

Validating the model


  0%|          | 0/6676 [00:00<?, ?it/s]

/home/beyond-data/anaconda3/envs/longt5/lib/python3.7/site-packages/transformers/modeling_utils.py:770: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning


Validation loss:  3.5737712138816224


ValueError: cannot set a row with mismatched columns

In [13]:
def model_validation(model,epoch, running_loss):
    bs = 12
    test_num_of_batches=len(test)/bs
    predictions = []
    torch.cuda.empty_cache()
    gc.collect()
    print("Validating the model")
    for i in tqdm_notebook(range(int(test_num_of_batches))):
        ib=[]
        lb = []
        new_df=test[i*bs:i*bs+bs]
        for indx,row in new_df.iterrows():
          input =row['input']
          ib.append(input)
          lb.append(row['Answer'])
          torch.cuda.empty_cache()
        ib=tokenizer.batch_encode_plus(ib,truncation=True,padding=True,max_length=4096,return_tensors='pt')["input_ids"].to(dev)
        outputs = model.generate(ib, do_sample=False,max_length=512,
                            min_length=50,num_return_sequences=1)
        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        for p in preds:
            predictions.append(p)
        torch.cuda.empty_cache()
        gc.collect()
    rouge = evaluate.load('rouge')
    references = test['Answer'][:len(predictions)]
    results = rouge.compute(predictions=predictions,
                         references=references,
                       rouge_types=['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_aggregator=False)
    m_p_r1 = 0
    m_r_r1 = 0
    result_row = []
    result_row.append(running_loss)
    result_row.append(epoch)
    for k in results.keys():
        p=[]
        r=[]
        f=[]
        for a in range(len(results[k])):
            p.append(results[k][a][0])
            r.append(results[k][a][1])
            f.append(results[k][a][2])
        m_p_r1 = sum(p)/len(p)
        m_r_r1 = sum(r)/len(r)
        m_f_r1 = sum(f)/len(f)
        print(m_p_r1, '-------',m_r_r1,'-------',m_f_r1)
        result_row.append(m_p_r1)
        result_row.append(m_r_r1)
        result_row.append(m_f_r1)
    result_df.loc[len(result_df)]=result_row[:11]
    result_df.to_csv("LongT5result.csv")